In [74]:
import torch
import torchvision
import torch.nn as nn
import numpy as np
import torch.utils.data as data
import torchvision.transforms as transforms
import torchvision.datasets as dsets
from torch.autograd import Variable

print("done importing")

done importing


# Basic autograd example 1

### create tensors

In [75]:
x=Variable(torch.Tensor([1]), requires_grad=True)
w=Variable(torch.Tensor([2]),requires_grad=True)
b=Variable(torch.Tensor([3]),requires_grad=True)


In [76]:
print(x)
print(w)
print(b)

Variable containing:
 1
[torch.FloatTensor of size 1]

Variable containing:
 2
[torch.FloatTensor of size 1]

Variable containing:
 3
[torch.FloatTensor of size 1]



### Build a computational graph

In [77]:
y = w*x + b
print("done")

done


In [78]:
y.backward()

print("Done Computing")

Done Computing


### Print Gradients

In [79]:
print(x.grad)
print(w.grad)
print(b.grad)

Variable containing:
 2
[torch.FloatTensor of size 1]

Variable containing:
 1
[torch.FloatTensor of size 1]

Variable containing:
 1
[torch.FloatTensor of size 1]



# Basic autograd example 2

### create tensors

In [80]:
x=Variable(torch.randn(5,3))
y=Variable(torch.randn(5,2))

print("created tensors")



created tensors


In [81]:
print(x)
print(y)

Variable containing:
-1.2988  1.5977  0.7689
-1.3407  0.4497 -2.5348
-0.3572  0.4662  0.5047
-0.6933 -0.4069  0.5110
-0.6746 -1.2283  0.8264
[torch.FloatTensor of size 5x3]

Variable containing:
-0.1829 -0.0954
-0.3225  0.0991
 0.1649  1.2503
 1.2825  0.8854
 0.3021 -1.5531
[torch.FloatTensor of size 5x2]



In [82]:
### build a linear Layers

In [83]:
linear = nn.Linear(3,2)

print("w:", linear.weight)
print("b:", linear.bias)


#


w: Parameter containing:
 0.1528  0.3210 -0.1755
-0.2990  0.1249  0.3159
[torch.FloatTensor of size 2x3]

b: Parameter containing:
-0.2647
-0.5517
[torch.FloatTensor of size 2]



In [84]:
#Build loss and Optimizer

criterion = nn.MSELoss()
print(criterion)
optimizer = torch.optim.SGD(linear.parameters(), lr=0.01)
print(optimizer)



MSELoss (
)


In [85]:
#forward propagation

pred=linear(x)
print("done")

done


In [86]:
#compute loss

loss =criterion(pred, y)
print('loss: ', loss.data[0])
print(loss)

#back propagation
loss.backward()

print("compute backpropagtion")


#print out the gradients.

print("dl/dw :", linear.weight.grad)
print("dl/db : ", linear.bias.grad)

loss:  1.1639149188995361
Variable containing:
 1.1639
[torch.FloatTensor of size 1]

compute backpropagtion
dl/dw : Variable containing:
 0.3092  0.4810 -0.6431
 0.2533 -0.3385  0.5149
[torch.FloatTensor of size 2x3]

dl/db :  Variable containing:
-0.5931
-0.3812
[torch.FloatTensor of size 2]



In [87]:
#one step Optimization (gradient descent)

optimizer.step()




print("optimize done")
#print out the loss after iptimization

pred = linear(x)
loss=criterion(pred,y)

print("loss after 1 step optimization: ", loss.data[0])

optimize done
loss after 1 step optimization:  1.1472301483154297


### Loading data from numpy

In [88]:
a= np.array([[1,2],[3,4]])
b=torch.from_numpy(a)
c=b.numpy()
print(a)
print(b)
print(c)


[[1 2]
 [3 4]]

 1  2
 3  4
[torch.LongTensor of size 2x2]

[[1 2]
 [3 4]]


### Implementing the input pipline

In [89]:
#Donwnload and construct dataset.

train_dataset=dsets.CIFAR10(root="../data/",
                           train=True,
                            transform=transforms.ToTensor(),
                            download=True
                           )


MemoryError: 

In [ ]:
#select one data pair(read data from disk).
image, label = train_dataset[0]

print(image.size())
print(label)

In [ ]:
#data loader
train_loader=torch.utils.data.DataLoader(dataset=train_dataset,
                                        batch_size=100,
                                        shuffle=True,
                                        num_workers=2)
print("done")

In [ ]:
#When iteration starts, queue and thread start tp load dataset from files.
data_iter= iter(train_loader)

print("done")

In [ ]:
#Mini batch images and labels

images, labels=data_iter.next()


#Actual usage of data loader is as below.

for images, labels in train_loader:
    # Your training code will be written here
    pass



### Using pretrained model


In [ ]:
#Download and load pretrained resnet.

resnet= torchvision.models.resnet18(pretrained=True)

#If you want to finetune only top layer of the model.

for param in resnet.parameters():
    param.requires_grad=False
    
#replace top layer for finetuning
resnet.fc = nn.Linear(resnet.fc.in_features, 100)
# For test

images= Variable(torch.randn(10,3,256,256))
print(images)
outputs= resnet(images)
print(outputs.size())



### save and load the model



In [ ]:
#save and load the entire model.

torch.save(resnet, 'model.pkl')
model= torch.load('model.pkl')



In [ ]:
#save and load only the model parameters(Recommended).

torch.save(resnet.state_dict(), 'params.pkl')
resnet.load_state_dict(torch.load('params.pkl'))